In [97]:
import urllib
from bs4 import BeautifulSoup as bs
import nltk
import pandas  as pd
import numpy as np
from nltk.stem.snowball import SnowballStemmer
from nltk import ngrams
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
df = pd.read_csv('news_summary.csv')

In [98]:
def clean_data(html,baseurl):
    soup = bs(html, 'html.parser')
    if baseurl == 'i':
        entries = soup.find_all('div',{'class':'description'})
    elif baseurl == 't':
        entries = soup.find_all('div',{'itemprop':'articleBody'})
    elif baseurl == 'h':
        entries = soup.find_all('div',{'itemprop':'articlebody'})
    for each in entries:
            if each.figure:
                each.figure.decompose()
    content = []
    for e in entries:
         content.extend(e.find_all("p"))
    
    text = ""
    for each in content:
        text = text + each.get_text() +" "

    text = text.encode('utf-8').decode("unicode_escape").encode('ascii','ignore')
#     text = nltk.sent_tokenize(text)    
    return text

In [99]:
def readUrl(url):
    if "indiatoday" in url or "intoday" in url:
        baseurl = "i"
    elif "hindustantimes" in url:
        baseurl = "h"
    elif "theguardian" in url:
        baseurl = "t"

    file = urllib.urlopen(url)
    html = file.read()

    text = clean_data(html,baseurl)
    return text

In [116]:
data = []
df1 = list(df['read_more'])
df1 = df1[:20]
len (df1)
for doc in df1:
#      print doc
     data.append(readUrl(doc))

In [117]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
       self.labels_list = labels_list
       self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc.split(),tags=[self.labels_list[idx]])

In [118]:
import gensim
from gensim.models.doc2vec import TaggedDocument, LabeledSentence, Doc2Vec

it = LabeledLineSentence(data, df1)

In [119]:
model = gensim.models.Doc2Vec(vector_size=300, window=10, min_count=5, workers=11,alpha=0.025, min_alpha=0.025) # use fixed learning rate

In [120]:
import sys
model.build_vocab(it)
for epoch in range(10):
    model.train(it,total_examples = model.corpus_count,epochs = model.epochs)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no deca

In [131]:
model.docvecs

In [ ]:
import numpy
from gensim.models import Word2Vec,Doc2Vec



w2v_model_filename = "../dataset/w2v_model"

class Sen2VecByWord2Vec(object):

    def __init__(self):
        self.w2v_model = Word2Vec.load(w2v_model_filename)

    def _w2v(self,word):
        try:
            return self.w2v_model[word]
        except:
            return None

    def sen2vec(self,sentence):
        distributed_words = []
        for word in sentence:
            ret = self._w2v(word)
            if ret is not None:
                distributed_words.append(ret)

        if (len(distributed_words) ):
            return numpy.array(distributed_words).astype("float32").mean(axis=0)
        else:
            return None

    def sens2vec(self,sentences):
        sens = []
        for sentence in sentences:
            sens.append(self.sen2vec(sentence))

        return sens


d2v_model_filename = "../dataset/d2v_model"

class Sen2VecByDoc2Vec(object):

    def __init__(self):
        self.d2v_model = Doc2Vec.load(d2v_model_filename)


    def sen2vec(self,sentence):
        return self.d2v_model.infer_vector(sentence,steps=100)

    def sens2vec(self,sentences):
        sens = []
        for i,sentence in enumerate(sentences):
            print str(i) + "\r",
            sens.append(self.sen2vec(sentence))
        print ""

return sens